In [1]:
import timm
import torchsummary
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from crd.criterion import CRDLoss
from munch import Munch
import yaml
import sys
from gd import Generator, Discriminator
from datetime import datetime
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# from dataset.imagenet import get_dataloader_sample
# from torchvision import transforms

In [2]:
# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.parallel import convert_syncbn_model
has_apex = False

In [3]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
if args.distributed:
    args.num_gpu = 1
    args.device = 'cuda:%d' % args.local_rank
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl', init_method='env://', rank=args.rank, world_size=args.world_size)
    args.world_size = torch.distributed.get_world_size()
    args.rank = torch.distributed.get_rank()
    assert args.rank >= 0

In [7]:
torch.manual_seed(args.seed + args.rank)

In [8]:
model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
model_ns = model_ns.cuda()
model_ns = torch.nn.DataParallel(model_ns)

In [9]:
model_raw = timm.create_model('tf_efficientnet_b1', pretrained=True)
model_raw = model_raw.cuda()
model_raw = torch.nn.DataParallel(model_raw)

In [10]:
model_g = Generator(args, img_size=240, max_conv_dim=256)
model_g = model_g.cuda()

In [11]:
optimizer = create_optimizer(args, model_g)

In [12]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    model_raw =  amp.initialize(model_raw)
    model_ns = amp.initialize(model_ns)
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX not installed. AMP off.


In [13]:
# optionally resume from a checkpoint
resume_state = {}
resume_epoch = None
if args.resume:
    resume_state, resume_epoch = resume_checkpoint(model_g, args.resume)
if resume_state and not args.no_resume_opt:
    if 'optimizer' in resume_state:
        if args.local_rank == 0:
            logging.info('Restoring Optimizer state from checkpoint')
        optimizer.load_state_dict(resume_state['optimizer'])
    if use_amp and 'amp' in resume_state and 'load_state_dict' in amp.__dict__:
        if args.local_rank == 0:
            logging.info('Restoring NVIDIA AMP state from checkpoint')
        amp.load_state_dict(resume_state['amp'])
del resume_state

In [14]:
model_g = torch.nn.DataParallel(model_g)

In [15]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model_g, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [16]:
num_aug_splits = 0

In [17]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP


In [18]:
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [19]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [20]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 210


In [21]:
train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [22]:
param_count = sum([m.numel() for m in model_g.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 4192780


In [23]:
collate_fn = None
if args.prefetcher and args.mixup > 0:
    assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
    collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [24]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)

In [25]:
train_loader = create_loader(
        train_dataset,
        input_size=data_config['input_size'],
        batch_size=args.batch_size,
        is_training=True,
        use_prefetcher=args.prefetcher,
        re_prob=args.reprob,
        re_mode=args.remode,
        re_count=args.recount,
        re_split=args.resplit,
        color_jitter=args.color_jitter,
        auto_augment=args.aa,
        num_aug_splits=num_aug_splits,
        interpolation=args.train_interpolation,
        mean=data_config['mean'],
        std=data_config['std'],
        num_workers=args.workers,
        distributed=args.distributed,
        collate_fn=collate_fn,
        pin_memory=args.pin_mem,
        use_multi_epochs_loader=args.use_multi_epochs_loader
    )

In [26]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [27]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.mixup > 0.:
    # smoothing is handled with mixup label transform
    train_loss_fn = SoftTargetCrossEntropy()
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
else:
    train_loss_fn = nn.CrossEntropyLoss()
    validate_loss_fn = train_loss_fn

crd_loss_fn = CRDLoss(args).cuda()

In [28]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [29]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [30]:
def train_epoch(epoch, model_g, model_raw, model_ns, loader, optimizer, loss_fn, crd_loss_fn, args,
               lr_scheduler=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_g = AverageMeter()
    losses_ce = AverageMeter()
    losses_kd = AverageMeter()
    
    model_g.train()
    model_ns.eval()
    model_raw.eval()

    real, fake = 1, 0
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
    for batch_idx, (inputs, target) in enumerate(loader):
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        z = torch.randn(args.batch_size, 1, 15, 15, device='cuda')
        g_out = model_g(z)
        inputs, target = inputs.cuda(), target.cuda()
        
        # KD train
        inputs_z = inputs + g_out
        output, traj_raw = model_raw(inputs_z)
        with torch.no_grad():
            out_ns, traj_ns = model_ns(inputs)
            out_ns = out_ns.detach()
            
        p_s = F.log_softmax(output/args.T, dim=1)
        p_t = F.softmax(out_ns/args.T, dim=1)
        loss_kd = F.kl_div(p_s, p_t, size_average=False) * (args.T ** 2) / output.shape[0]
        
        # CE train
        loss_ce = loss_fn(output, target)
        
        # overall loss
        loss = args.lambda_kd * loss_kd + args.lambda_ce * loss_ce
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not args.distributed:
#             losses_d.update(loss_d.item(), inputs.size(0))
#             losses_g.update(loss_g.item(), inputs.size(0))
            losses_kd.update(loss_kd.item(), inputs.size(0))
            losses_ce.update(loss_ce.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))

        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
#                     'Loss_d: {loss_d.val:>9.6f} ({loss_d.avg:>6.4f})  '
#                     'Loss_g: {loss_g.val:>9.6f} ({loss_g.avg:>6.4f})  '
                    'Loss_kd: {loss_kd.val:>9.6f} ({loss_kd.avg:>6.4f})  '
                    'Loss_ce: {loss_ce.val:>9.6f} ({loss_ce.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.3e}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
#                         loss_d=losses_d,
#                         loss_g=losses_g,
                        loss_kd=losses_kd,
                        loss_ce=losses_ce,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)

        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()

    return OrderedDict([('loss', losses_m.avg)])

In [31]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()
            z = torch.randn(inputs.shape[0], 1, 15, 15, device='cuda')
            
            out = model(z)
            # synthesizing input + generator
            inputs_out = inputs + out
            # compute output
            output, foward_list = model_raw(inputs_out)
            loss = criterion(output, target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [32]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        train_loader.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model_g, model_raw, model_ns, train_loader, optimizer, train_loss_fn, crd_loss_fn, args,
        lr_scheduler=lr_scheduler, saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    eval_metrics = val_epoch(model_raw, model_g, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')


    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])
        
    update_summary(
        epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
        write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model_g, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)

/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
INFO:root:Train: 10 [   0/5546 (  0%)]  Loss: 11.796969 (11.7970)  Loss_kd:  0.793030 (0.7930)  Loss_ce:  3.866670 (3.8667)  Time: 22.373s,   30.98/s  (22.373s,   30.98/s)  LR: 9.939e-04  Data: 4.702 (4.702)
INFO:root:Train: 10 [  50/5546 (  1%)]  Loss:  5.170341 (5.5878)  Loss_kd:  0.349773 (0.3767)  Loss_ce:  1.672613 (1.8206)  Time: 1.219s,  568.40/s  (1.751s,  395.71/s)  LR: 9.939e-04  Data: 0.001 (0.093)
INFO:root:Train: 10 [ 100/5546 (  2%)]  Loss:  5.241355 (5.4384)  Loss_kd:  0.353239 (0.3657)  Loss_ce:  1.708962 (1.7811)  Time: 1.223s,  566.53/s  (1.491s,  464.81/s)  LR: 9.939e-04  Data: 0.001 (0.048)
INFO:root:Train: 10 [ 150/5546 (  3%)]  Loss:  5.369418 (5.3907)  Loss_kd:  0.363769 (0.3618)  Loss_ce:  1.731726 (1.7727)  Time: 1.272s,  544.78/s  (1.402s,  494.19/s)  L

INFO:root:Train: 10 [1950/5546 ( 35%)]  Loss:  5.537895 (5.2775)  Loss_kd:  0.368515 (0.3541)  Loss_ce:  1.852745 (1.7365)  Time: 1.220s,  568.14/s  (1.237s,  560.03/s)  LR: 9.939e-04  Data: 0.001 (0.003)
INFO:root:Train: 10 [2000/5546 ( 36%)]  Loss:  5.185229 (5.2774)  Loss_kd:  0.340785 (0.3541)  Loss_ce:  1.777378 (1.7364)  Time: 1.215s,  570.16/s  (1.237s,  560.19/s)  LR: 9.939e-04  Data: 0.001 (0.003)
INFO:root:Train: 10 [2050/5546 ( 37%)]  Loss:  5.334477 (5.2771)  Loss_kd:  0.350060 (0.3541)  Loss_ce:  1.833881 (1.7364)  Time: 1.223s,  566.69/s  (1.237s,  560.35/s)  LR: 9.939e-04  Data: 0.001 (0.003)
INFO:root:Train: 10 [2100/5546 ( 38%)]  Loss:  5.224851 (5.2773)  Loss_kd:  0.348657 (0.3541)  Loss_ce:  1.738284 (1.7364)  Time: 1.218s,  569.06/s  (1.236s,  560.51/s)  LR: 9.939e-04  Data: 0.001 (0.003)
INFO:root:Train: 10 [2150/5546 ( 39%)]  Loss:  5.477279 (5.2780)  Loss_kd:  0.363293 (0.3541)  Loss_ce:  1.844353 (1.7368)  Time: 1.216s,  570.00/s  (1.236s,  560.64/s)  LR: 9.939e

INFO:root:Train: 10 [3850/5546 ( 69%)]  Loss:  5.379971 (5.2723)  Loss_kd:  0.361651 (0.3538)  Loss_ce:  1.763458 (1.7346)  Time: 1.219s,  568.48/s  (1.231s,  563.18/s)  LR: 9.939e-04  Data: 0.001 (0.002)
INFO:root:Train: 10 [3900/5546 ( 70%)]  Loss:  5.292000 (5.2724)  Loss_kd:  0.355853 (0.3538)  Loss_ce:  1.733465 (1.7344)  Time: 1.219s,  568.65/s  (1.230s,  563.21/s)  LR: 9.939e-04  Data: 0.001 (0.002)
INFO:root:Train: 10 [3950/5546 ( 71%)]  Loss:  5.245749 (5.2724)  Loss_kd:  0.349544 (0.3538)  Loss_ce:  1.750308 (1.7345)  Time: 1.220s,  567.82/s  (1.230s,  563.26/s)  LR: 9.939e-04  Data: 0.001 (0.002)
INFO:root:Train: 10 [4000/5546 ( 72%)]  Loss:  5.139197 (5.2724)  Loss_kd:  0.348581 (0.3538)  Loss_ce:  1.653386 (1.7344)  Time: 1.220s,  568.22/s  (1.230s,  563.28/s)  LR: 9.939e-04  Data: 0.001 (0.002)
INFO:root:Train: 10 [4050/5546 ( 73%)]  Loss:  5.241813 (5.2720)  Loss_kd:  0.352560 (0.3538)  Loss_ce:  1.716210 (1.7344)  Time: 1.216s,  570.09/s  (1.230s,  563.32/s)  LR: 9.939e

INFO:root:Test: [  80/217]  Time: 0.358s (0.433s,  533.34/s)  Loss:  0.8847 (0.6705)  Acc@1:  83.117 ( 84.196)  Acc@5:  95.238 ( 96.799)
INFO:root:Test: [  90/217]  Time: 0.357s (0.425s,  543.36/s)  Loss:  0.8275 (0.6948)  Acc@1:  83.117 ( 83.626)  Acc@5:  96.537 ( 96.513)
INFO:root:Test: [ 100/217]  Time: 0.357s (0.419s,  551.66/s)  Loss:  1.6199 (0.7324)  Acc@1:  62.338 ( 82.701)  Acc@5:  84.848 ( 96.065)
INFO:root:Test: [ 110/217]  Time: 0.357s (0.413s,  558.95/s)  Loss:  0.8299 (0.7724)  Acc@1:  80.519 ( 81.721)  Acc@5:  93.939 ( 95.612)
INFO:root:Test: [ 120/217]  Time: 0.354s (0.409s,  565.29/s)  Loss:  1.3079 (0.7944)  Acc@1:  72.727 ( 81.210)  Acc@5:  90.043 ( 95.406)
INFO:root:Test: [ 130/217]  Time: 0.354s (0.405s,  570.54/s)  Loss:  0.9276 (0.8030)  Acc@1:  80.087 ( 81.104)  Acc@5:  93.074 ( 95.281)
INFO:root:Test: [ 140/217]  Time: 0.354s (0.402s,  575.27/s)  Loss:  1.2746 (0.8231)  Acc@1:  73.160 ( 80.627)  Acc@5:  89.177 ( 94.992)
INFO:root:Test: [ 150/217]  Time: 0.360s 

INFO:root:Train: 11 [1500/5546 ( 27%)]  Loss:  5.346499 (5.2655)  Loss_kd:  0.353850 (0.3535)  Loss_ce:  1.808004 (1.7303)  Time: 1.215s,  570.35/s  (1.228s,  564.19/s)  LR: 9.926e-04  Data: 0.001 (0.004)
INFO:root:Train: 11 [1550/5546 ( 28%)]  Loss:  5.269265 (5.2646)  Loss_kd:  0.350216 (0.3535)  Loss_ce:  1.767106 (1.7297)  Time: 1.218s,  568.75/s  (1.228s,  564.30/s)  LR: 9.926e-04  Data: 0.001 (0.004)
INFO:root:Train: 11 [1600/5546 ( 29%)]  Loss:  5.066825 (5.2644)  Loss_kd:  0.343023 (0.3535)  Loss_ce:  1.636595 (1.7299)  Time: 1.216s,  569.94/s  (1.228s,  564.35/s)  LR: 9.926e-04  Data: 0.001 (0.004)
INFO:root:Train: 11 [1650/5546 ( 30%)]  Loss:  5.261494 (5.2649)  Loss_kd:  0.347875 (0.3535)  Loss_ce:  1.782740 (1.7303)  Time: 1.265s,  547.62/s  (1.228s,  564.43/s)  LR: 9.926e-04  Data: 0.001 (0.004)
INFO:root:Train: 11 [1700/5546 ( 31%)]  Loss:  5.052326 (5.2655)  Loss_kd:  0.339757 (0.3534)  Loss_ce:  1.654760 (1.7311)  Time: 1.218s,  568.89/s  (1.228s,  564.48/s)  LR: 9.926e

...

INFO:root:Train: 40 [3550/5546 ( 64%)]  Loss:  5.254851 (5.2524)  Loss_kd:  0.351896 (0.3523)  Loss_ce:  1.735893 (1.7290)  Time: 1.224s,  566.28/s  (1.226s,  565.09/s)  LR: 9.055e-04  Data: 0.005 (0.003)
INFO:root:Train: 40 [3600/5546 ( 65%)]  Loss:  5.049375 (5.2519)  Loss_kd:  0.343068 (0.3523)  Loss_ce:  1.618698 (1.7288)  Time: 1.215s,  570.47/s  (1.226s,  565.11/s)  LR: 9.055e-04  Data: 0.001 (0.003)
INFO:root:Train: 40 [3650/5546 ( 66%)]  Loss:  5.217284 (5.2522)  Loss_kd:  0.364526 (0.3523)  Loss_ce:  1.572021 (1.7289)  Time: 1.219s,  568.40/s  (1.226s,  565.11/s)  LR: 9.055e-04  Data: 0.001 (0.003)
INFO:root:Train: 40 [3700/5546 ( 67%)]  Loss:  5.270127 (5.2524)  Loss_kd:  0.362739 (0.3523)  Loss_ce:  1.642741 (1.7290)  Time: 1.220s,  568.10/s  (1.226s,  565.12/s)  LR: 9.055e-04  Data: 0.001 (0.003)
INFO:root:Train: 40 [3750/5546 ( 68%)]  Loss:  5.330215 (5.2525)  Loss_kd:  0.364409 (0.3524)  Loss_ce:  1.686124 (1.7290)  Time: 1.221s,  567.65/s  (1.226s,  565.13/s)  LR: 9.055e

INFO:root:Train: 40 [5545/5546 (100%)]  Loss:  5.008136 (5.2522)  Loss_kd:  0.342799 (0.3524)  Loss_ce:  1.580143 (1.7287)  Time: 1.653s,  419.12/s  (1.226s,  565.34/s)  LR: 9.055e-04  Data: 0.440 (0.002)
INFO:root:Test: [   0/217]  Time: 6.687s (6.687s,   34.54/s)  Loss:  0.5031 (0.5031)  Acc@1:  88.312 ( 88.312)  Acc@5:  97.835 ( 97.835)
INFO:root:Test: [  10/217]  Time: 0.359s (0.939s,  245.96/s)  Loss:  0.7968 (0.6356)  Acc@1:  81.818 ( 85.242)  Acc@5:  95.671 ( 97.048)
INFO:root:Test: [  20/217]  Time: 0.359s (0.665s,  347.41/s)  Loss:  0.3247 (0.6846)  Acc@1:  93.506 ( 83.797)  Acc@5:  98.268 ( 96.619)
INFO:root:Test: [  30/217]  Time: 0.364s (0.567s,  407.34/s)  Loss:  0.3269 (0.6444)  Acc@1:  93.506 ( 85.086)  Acc@5:  98.268 ( 96.635)
INFO:root:Test: [  40/217]  Time: 0.356s (0.517s,  447.10/s)  Loss:  0.9621 (0.6702)  Acc@1:  74.459 ( 84.299)  Acc@5:  95.671 ( 96.579)
INFO:root:Test: [  50/217]  Time: 0.356s (0.485s,  475.97/s)  Loss:  0.7588 (0.6806)  Acc@1:  79.654 ( 84.000)

INFO:root:Train: 41 [1100/5546 ( 20%)]  Loss:  5.211532 (5.2458)  Loss_kd:  0.343857 (0.3520)  Loss_ce:  1.772959 (1.7260)  Time: 1.220s,  568.11/s  (1.231s,  563.06/s)  LR: 9.008e-04  Data: 0.001 (0.006)
INFO:root:Train: 41 [1150/5546 ( 21%)]  Loss:  5.152231 (5.2464)  Loss_kd:  0.344040 (0.3520)  Loss_ce:  1.711826 (1.7260)  Time: 1.219s,  568.64/s  (1.231s,  563.18/s)  LR: 9.008e-04  Data: 0.001 (0.006)
INFO:root:Train: 41 [1200/5546 ( 22%)]  Loss:  5.327911 (5.2458)  Loss_kd:  0.348616 (0.3520)  Loss_ce:  1.841755 (1.7257)  Time: 1.219s,  568.30/s  (1.230s,  563.32/s)  LR: 9.008e-04  Data: 0.001 (0.006)
INFO:root:Train: 41 [1250/5546 ( 23%)]  Loss:  5.080624 (5.2459)  Loss_kd:  0.340641 (0.3520)  Loss_ce:  1.674214 (1.7258)  Time: 1.279s,  541.89/s  (1.230s,  563.42/s)  LR: 9.008e-04  Data: 0.001 (0.005)
INFO:root:Train: 41 [1300/5546 ( 23%)]  Loss:  4.967809 (5.2464)  Loss_kd:  0.333538 (0.3520)  Loss_ce:  1.632428 (1.7264)  Time: 1.220s,  568.03/s  (1.230s,  563.56/s)  LR: 9.008e

INFO:root:Train: 41 [3100/5546 ( 56%)]  Loss:  5.014387 (5.2500)  Loss_kd:  0.340032 (0.3523)  Loss_ce:  1.614070 (1.7274)  Time: 1.222s,  567.09/s  (1.226s,  565.11/s)  LR: 9.008e-04  Data: 0.001 (0.003)
INFO:root:Train: 41 [3150/5546 ( 57%)]  Loss:  5.287643 (5.2499)  Loss_kd:  0.361598 (0.3523)  Loss_ce:  1.671660 (1.7273)  Time: 1.218s,  568.75/s  (1.226s,  565.12/s)  LR: 9.008e-04  Data: 0.001 (0.003)
INFO:root:Train: 41 [3200/5546 ( 58%)]  Loss:  5.444836 (5.2500)  Loss_kd:  0.363413 (0.3523)  Loss_ce:  1.810705 (1.7273)  Time: 1.222s,  567.24/s  (1.226s,  565.14/s)  LR: 9.008e-04  Data: 0.001 (0.003)
INFO:root:Train: 41 [3250/5546 ( 59%)]  Loss:  5.218083 (5.2498)  Loss_kd:  0.348289 (0.3523)  Loss_ce:  1.735194 (1.7272)  Time: 1.216s,  569.77/s  (1.226s,  565.15/s)  LR: 9.008e-04  Data: 0.001 (0.003)
INFO:root:Train: 41 [3300/5546 ( 60%)]  Loss:  5.209189 (5.2501)  Loss_kd:  0.356333 (0.3523)  Loss_ce:  1.645857 (1.7274)  Time: 1.269s,  546.10/s  (1.226s,  565.16/s)  LR: 9.008e

KeyboardInterrupt: 